In [1]:
#import essential packages 

import pandas as pd
import numpy as np
import json
from pandas.io.json import json_normalize 


In [2]:
#read data using Pandas as JSON file

data= pd.read_json('data/world_bank_projects.json')

In [3]:
#check DataFrame to get some initial insight of it 

data.info()
data.head(4)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 500 entries, 0 to 499
Data columns (total 50 columns):
_id                         500 non-null object
approvalfy                  500 non-null int64
board_approval_month        500 non-null object
boardapprovaldate           500 non-null object
borrower                    485 non-null object
closingdate                 370 non-null object
country_namecode            500 non-null object
countrycode                 500 non-null object
countryname                 500 non-null object
countryshortname            500 non-null object
docty                       446 non-null object
envassesmentcategorycode    430 non-null object
grantamt                    500 non-null int64
ibrdcommamt                 500 non-null int64
id                          500 non-null object
idacommamt                  500 non-null int64
impagency                   472 non-null object
lendinginstr                495 non-null object
lendinginstrtype            495 non

,_id,approvalfy,board_approval_month,boardapprovaldate,borrower,closingdate,country_namecode,countrycode,countryname,countryshortname,...,sectorcode,source,status,supplementprojectflg,theme1,theme_namecode,themecode,totalamt,totalcommamt,url
0,{'$oid': '52b213b38594d8a2be17c780'},1999,November,2013-11-12T00:00:00Z,FEDERAL DEMOCRATIC REPUBLIC OF ETHIOPIA,2018-07-07T00:00:00Z,Federal Democratic Republic of Ethiopia!$!ET,ET,Federal Democratic Republic of Ethiopia,Ethiopia,...,"ET,BS,ES,EP",IBRD,Active,N,"{'Percent': 100, 'Name': 'Education for all'}","[{'code': '65', 'name': 'Education for all'}]",65,130000000,130000000,http://www.worldbank.org/projects/P129828/ethi...
1,{'$oid': '52b213b38594d8a2be17c781'},2015,November,2013-11-04T00:00:00Z,GOVERNMENT OF TUNISIA,NaN,Republic of Tunisia!$!TN,TN,Republic of Tunisia,Tunisia,...,"BZ,BS",IBRD,Active,N,"{'Percent': 30, 'Name': 'Other economic manage...","[{'code': '24', 'name': 'Other economic manage...","54,24",0,4700000,http://www.worldbank.org/projects/P144674?lang=en
2,{'$oid': '52b213b38594d8a2be17c782'},2014,November,2013-11-01T00:00:00Z,MINISTRY OF FINANCE AND ECONOMIC DEVEL,NaN,Tuvalu!$!TV,TV,Tuvalu,Tuvalu,...,TI,IBRD,Active,Y,"{'Percent': 46, 'Name': 'Regional integration'}","[{'code': '47', 'name': 'Regional integration'...","52,81,25,47",6060000,6060000,http://www.worldbank.org/projects/P145310?lang=en
3,{'$oid': '52b213b38594d8a2be17c783'},2014,October,2013-10-31T00:00:00Z,MIN. OF PLANNING AND INT'L COOPERATION,NaN,Republic of Yemen!$!RY,RY,Republic of Yemen,"Yemen, Republic of",...,JB,IBRD,Active,N,"{'Percent': 50, 'Name': 'Participation and civ...","[{'code': '57', 'name': 'Participation and civ...","59,57",0,1500000,http://www.worldbank.org/projects/P144665?lang=en


### 1 - Find the 10 counties with the most projects : 

In [4]:
#countries with the most projects -> looking into coloumn:'countryname'

data_grp_country_count = data.groupby('countryname').size().sort_values(ascending = False )
print (data_grp_country_count.head(10))

countryname
People's Republic of China         19
Republic of Indonesia              19
Socialist Republic of Vietnam      17
Republic of India                  16
Republic of Yemen                  13
Nepal                              12
People's Republic of Bangladesh    12
Kingdom of Morocco                 12
Africa                             11
Republic of Mozambique             11
dtype: int64


### 2 - Find the most common project themes :

In [5]:
#the most common project themes -> we need to load our raw data as json_string in order to use json_normalize 

data_string = json.load((open('data/world_bank_projects.json')))
data_themes = json_normalize(data_string, 'mjtheme_namecode')
print (data_themes.head(10))

data_themes_group_name = data_themes.groupby('name').size().sort_values(ascending=False)
print (data_themes_group_name.head(10))

  code                                          name
0    8                             Human development
1   11                                              
2    1                           Economic management
3    6         Social protection and risk management
4    5                         Trade and integration
5    2                      Public sector governance
6   11  Environment and natural resources management
7    6         Social protection and risk management
8    7                   Social dev/gender/inclusion
9    7                   Social dev/gender/inclusion
name
Environment and natural resources management    223
Rural development                               202
Human development                               197
Public sector governance                        184
Social protection and risk management           158
Financial and private sector development        130
                                                122
Social dev/gender/inclusion                     

As you can see in the first table, there are some projects which have codes but the name is missing, so for getting the most accurate answer about the most common projects we have to fill the "name" column with the appropriate entries based on relavent "code". 


### 3 - In 2. above you will notice that some entries have only the code and the name is missing. Create a dataframe with the missing names filled in.

In [6]:
#Create a table of projects' codes and names without any missing value

# Fill '' with NaN

data_themes = data_themes.replace(to_replace='' , value=np.NaN)

# Sort Data frame based on the code and name and using "fillna" to replace None with Appropriate "Name"

data_themes_sorted_code = data_themes.sort_values(['code','name'],ascending = True)
clean_data=data_themes_sorted_code['name'].fillna(inplace=True, method='ffill')


# Get the most common project themes based on the new changes 

data_themes_clean_grpby_name = data_themes_sorted_code.groupby(['name','code']).size().sort_values(ascending=False)
print (data_themes_clean_grpby_name.head(10))

name                                          code
Environment and natural resources management  11      250
Rural development                             10      216
Human development                             8       210
Public sector governance                      2       199
Social protection and risk management         6       168
Financial and private sector development      4       146
Social dev/gender/inclusion                   7       130
Trade and integration                         5        77
Urban development                             9        50
Economic management                           1        38
dtype: int64


So as you can see after cleaning the data set and filling the missing project names based on their relavent codes, we got the different number of projects (accurate number) for each project theme. 